In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import seaborn as sns
from scipy.stats import norm, pearsonr, spearmanr
import scipy.stats as stats
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #last_expr

In [2]:
kos_res = pd.read_csv('KO差异分析.tsv', sep='\t', index_col=0)
kos_res.head(2)
path2ko = pd.read_csv('KEGG_path2ko.csv', index_col=0)
path2ko.head(2)
path2name = pd.read_csv('KEGG_pathways.csv', index_col=0)
path2name.head(2)

,ID,KO,N_control,N_disease,AveExpr_control,AveExpr_disease,FC,P.Value
K11690,K11690,"K11690: C4-dicarboxylate transporter, DctM sub...",72,14,0.000223,0.000057,0.254282,0.000047
K19510,K19510,K19510: NO_NAME,72,14,0.000086,0.000005,0.054256,0.000057


,Pathway,KO
map00515,map00515,K00728
map00515,map00515,K00735


,AClass,BClass,ID,Name
map00515,Metabolism,Glycan biosynthesis and metabolism,map00515,Mannose type O-glycan biosynthesis
map05130,Human Diseases,Infectious disease: bacterial,map05130,Pathogenic Escherichia coli infection


### 1. Z score of KOs
* inverse normal cumulative distribution (norm.ppf)
* performed on all the KOs that occurred in more than five samples and adjusted for multiple testing using the Benjamin-Hochberg procedure. 

In [3]:
zscores = []
for i in kos_res['P.Value'].clip(0, 0.99999):
    zscores.append(norm.ppf(1-i))
kos_res['Zscore'] = zscores
kos_res.head(5)

,ID,KO,N_control,N_disease,AveExpr_control,AveExpr_disease,FC,P.Value,Zscore
K11690,K11690,"K11690: C4-dicarboxylate transporter, DctM sub...",72,14,0.000223,0.000057,0.254282,0.000047,3.906094
K19510,K19510,K19510: NO_NAME,72,14,0.000086,0.000005,0.054256,0.000057,3.857826
K02278,K02278,K02278: prepilin peptidase CpaA,72,14,0.000204,0.000048,0.235508,0.000062,3.839670
K01209,K01209,K01209: alpha-N-arabinofuranosidase [EC:3.2.1.55],72,14,0.000619,0.000179,0.289785,0.000077,3.784852
K02172,K02172,K02172: bla regulator protein blaR1,72,14,0.000135,0.000031,0.229791,0.000077,3.784852


### 2. Aggregated Z-score for a KEGG pathway (or module)
* A reporterscore of Z>=1.6 (90% confidence according to normal distribution) could be used as a detection threshold for significantly differentiating pathways.
* Absolute value of reporter score = 1.6 or higher (95% confidence on either tail, according to normal distribution) could be used as a detection threshold for significantly differentiating pathways.

In [4]:
agg_res = []
for cluster in set(path2ko.index):
    raw_kos = path2ko.loc[cluster, 'KO']
    kos = set(raw_kos)&set(kos_res.index)
    if len(kos)>=3:
        zcluster = 1/np.sqrt(len(kos))*sum(kos_res.loc[kos, 'Zscore'])
        ### backgroud
        bg = []
        for i in range(50):
            random_kos = np.random.choice(kos_res.index, len(kos))
            random_zcluster = 1/np.sqrt(len(random_kos))*sum(kos_res.loc[random_kos, 'Zscore'])
            bg.append(random_zcluster)
            #cluster, len(random_kos), random_zcluster
        bg = np.array(bg)
        zadjust = (zcluster-bg.mean())/bg.std()
        p = (bg>=zcluster).sum()/50.0
        p = p if p<=0.5 else 1-p
        agg_res.append([cluster, path2name.loc[cluster, 'Name'], len(raw_kos), len(kos), zcluster, bg.mean(), 
                        bg.std(), zadjust, p])
agg_res = pd.DataFrame(agg_res, columns=['ID', 'Module', 'KOs', 'MapKOs', 'Zscore',
                                         'BGMean', 'BGStd', 'Zadjust', 'p'])
agg_res

,ID,Module,KOs,MapKOs,Zscore,BGMean,BGStd,Zadjust,p
0,map02060,Phosphotransferase system (PTS),71,58,3.570428,0.401382,1.063092,2.980972,0.00
1,map05017,Spinocerebellar ataxia,116,49,-2.827138,0.676111,0.994906,-3.521185,0.00
2,map00590,Arachidonic acid metabolism,45,5,-0.355506,0.072935,1.108956,-0.386346,0.46
3,map05203,Viral carcinogenesis,174,32,-2.777802,0.453945,1.052371,-3.070920,0.00
4,map04391,Hippo signaling pathway - fly,53,8,-0.992976,0.183835,1.042323,-1.129027,0.16
...,...,...,...,...,...,...,...,...,...
362,map00010,Glycolysis / Gluconeogenesis,105,70,2.666124,0.800466,1.113848,1.674965,0.06
363,map05416,Viral myocarditis,39,3,-0.731866,-0.227991,1.025536,-0.491328,0.30
364,map04016,MAPK signaling pathway - plant,54,4,-1.063150,0.215954,1.004991,-1.272752,0.10
365,map05130,Pathogenic Escherichia coli infection,152,30,-1.842482,0.358242,1.144005,-1.923701,0.02


In [5]:
agg_res.loc[np.abs(agg_res['Zadjust'])>=1.64, :].sort_values(['Zadjust'], ascending=False)

,ID,Module,KOs,MapKOs,Zscore,BGMean,BGStd,Zadjust,p
352,map02040,Flagellar assembly,55,45,9.981351,0.683801,1.081656,8.595666,0.0
100,map02010,ABC transporters,515,311,7.353223,1.184222,0.821042,7.513628,0.0
209,map02030,Bacterial chemotaxis,26,25,7.660055,0.246312,1.153211,6.428780,0.0
181,map01240,Biosynthesis of cofactors,375,254,5.718064,1.431068,0.971603,4.412289,0.0
313,map04122,Sulfur relay system,29,21,4.512059,0.552388,0.980983,4.036431,0.0
...,...,...,...,...,...,...,...,...,...
274,map05020,Prion disease,209,93,-5.337923,1.007174,0.977595,-6.490518,0.0
327,map03010,Ribosome,143,134,-6.719248,0.914106,1.167131,-6.540273,0.0
251,map05022,Pathways of neurodegeneration - multiple diseases,368,115,-5.120378,1.147596,0.945053,-6.632408,0.0
330,map05016,Huntington disease,229,107,-5.509973,0.888790,0.945411,-6.768232,0.0


In [6]:
agg_res.to_csv('reporterscore.csv')